In [1]:
import torch
import time


tensor1 = torch.randint(0,1000,(64,1024,1024))
print(tensor1.shape)
tensor2 = torch.randint(0,1000,(128,512,512))
print(tensor2.shape)
tensor3 = torch.randint(0,1000,(256,256,256))
print(tensor3.shape)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется устройство: {device.upper()}")
tensor1_cpu = torch.rand((16, 512, 512))
tensor2_cpu = torch.rand((16, 512, 512))
tensor3_cpu = torch.rand((16, 512, 512))



# измерение времени
def timetable(operation, device):
    if device == 'cuda':
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()
    else:
        start_time = time.time()
    
    result = operation()

    if device == 'cuda':
        end_event.record()
        torch.cuda.synchronize()
        elapsed_time = start_event.elapsed_time(end_event) / 1000  
    else:
        end_time = time.time()
        elapsed_time = end_time - start_time

    return result, elapsed_time

#все оперцаии

operations = {
    "Матричное умножение": lambda t1, t2: lambda: torch.matmul(t1, t2),
    "Сложение": lambda t1, t2: lambda: t1 + t2,
    "Умножение": lambda t1, t2: lambda: t1 * t2,
    "Транспонирование": lambda t1, _: lambda: t1.transpose(1, 2),
    "Сумма всех элементов": lambda t1, _: lambda: torch.sum(t1)
}

# Таблица результатов
results = []

for name, op_func in operations.items():
    

    # --- На CPU ---
  
    _, cpu_time = timetable(op_func(tensor1_cpu, tensor2_cpu), 'cpu')
    cpu_ms = cpu_time * 1000

    # --- На GPU ---
    if device == 'cuda':
        tensor1_gpu = tensor1_cpu.cuda()
        tensor2_gpu = tensor2_cpu.cuda()
        _, gpu_time = timetable(op_func(tensor1_gpu, tensor2_gpu), 'cuda')
        gpu_ms = gpu_time * 1000
        speedup = cpu_ms / gpu_ms if gpu_ms > 0 else float('inf')
    

    results.append((name, cpu_ms, gpu_ms, speedup))

# --- Вывод таблицы ---
print("\n")
print("{:<20} | {:>8} | {:>8} | {:>10}".format("Операция", "CPU (мс)", "GPU (мс)", "Ускорение"))
print("-" * 60)
for row in results:
    name, cpu_ms, gpu_ms, speedup = row
    if isinstance(gpu_ms, str) or isinstance(speedup, str):
        print(f"{name:<20} | {cpu_ms:>8.1f} | {gpu_ms:>8} | {speedup:>10}")
    else:
        print(f"{name:<20} | {cpu_ms:>8.1f} | {gpu_ms:>8.1f} | {speedup:>9.1f}x")


''' Анализ данных:
исходя из полученной таблицы можно сделать выводы,что gpu работает лучше с большим набором данных и на сложных операциях,
 на легких операциях и на мелньких данных разницы почти нет или иногда чуть лучше cpu'''

torch.Size([64, 1024, 1024])
torch.Size([128, 512, 512])
torch.Size([256, 256, 256])
Используется устройство: CUDA


Операция             | CPU (мс) | GPU (мс) |  Ускорение
------------------------------------------------------------
Матричное умножение  |     14.0 |      6.3 |       2.2x
Сложение             |      3.0 |      3.3 |       0.9x
Умножение            |      3.0 |      0.4 |       7.3x
Транспонирование     |      0.0 |      0.0 |       0.0x
Сумма всех элементов |      1.0 |      0.3 |       2.9x


' Анализ данных:\nисходя из полученной таблицы можно сделать выводы,что gpu работает лучше с большим набором данных и на сложных операциях,\n на легких операциях и на мелньких данных разницы почти нет или иногда чуть лучше cpu'

In [ ]:
#3.4 Анализ результатов
